In [ ]:
import math

# Third-party
import astropy.units as u
from astropy.constants import G
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
import numpy as np
%matplotlib inline
import scipy.optimize as so
from scipy.integrate import quad
from scipy.special import erfc

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

import biff.scf as bscf

In [ ]:
def f(x, y, Rmax):
    R_2 = x**2 + y**2
    R0 = 0.5 # Dwek's number
    if R_2 < Rmax**2:
        return 1.
    else:
        return math.exp(-0.5 * R_2 / R0**2)
    
def f_arr(x, y, Rmax):
    x = np.array(x)
    y = np.array(y)
    
    R_2 = x**2 + y**2
    R0 = 0.5 # Dwek's number
    
    ret = np.zeros_like(x)
    mask = R_2 < Rmax**2
    ret[mask] = 1.
    ret[~mask] = np.exp(-0.5 * R_2[~mask] / R0**2)
    return ret

def dwek1995_G2_helper(x, y, z, x0, y0, z0, Rmax, lib, f):
    r1 = ( ((x/x0)**2 + (y/y0)**2)**2 + (z/z0)**4 ) ** 0.25
    return lib.exp(-r1**2/2.) * f(x, y, Rmax)
    
def dwek1995_G2_density(x, y, z, x0, y0, z0, Rmax): 
    return dwek1995_G2_helper(x, y, z, x0, y0, z0, Rmax, lib=math, f=f)

def dwek1995_G2_density_arr(x, y, z, x0, y0, z0, Rmax): 
    return dwek1995_G2_helper(x, y, z, x0, y0, z0, Rmax, lib=np, f=f_arr)

density = dwek1995_G2_density
density_arr = dwek1995_G2_density_arr
args = (1.49, 0.58, 0.4, 100.)

In [ ]:
def analytic_grad_on_axis(x, x0):
    # Only works along x, y, or z
    # G = rho0 = 1
    return (2*np.pi)**1.5 * x0 * erfc(x/x0/np.sqrt(2))

---

## Plotting the analytic density and computed surface density

In [ ]:
def surf_dens_helper(z, q, density, axis=0):
    return density(q, z)

In [ ]:
xs = np.linspace(0.001, 10, 64)
surf_dens_x = np.zeros_like(xs)
surf_dens_y = np.zeros_like(xs)

_density_x = lambda q, z: density(q, 0, z, *args)
_density_y = lambda q, z: density(0, q, z, *args)

for i,x in enumerate(xs):
    surf_dens_x[i],_ = quad(surf_dens_helper, 
                            -10, 10,
                            args=(x, _density_x))

    surf_dens_y[i],_ = quad(surf_dens_helper, 
                            -10, 10,
                            args=(x, _density_y))

In [ ]:
plt.figure(figsize=(5,5))

xyz = np.zeros((3, 1024))
xyz[0] = np.logspace(-3, 1., xyz.shape[1])

analytic_dens_x = np.array([density(x, 0., 0., *args) for x in xyz[0]])
plt.semilogy(xyz[0], analytic_dens_x, marker='', linestyle='-')
plt.xlim(xyz[0].min(), xyz[0].max())
plt.ylim(1E-6, 1E2)

plt.xlabel(r"$r$ [kpc]")
plt.ylabel(r"$\rho(r)$")

Compare this to Fig. 3 from Portail+2017:

<img src="Portail17_Fig3.png" width=750 />

In [ ]:
plt.figure(figsize=(5,5))

plt.semilogy(xs, surf_dens_x, marker='', linestyle='-', color='tab:red')
plt.semilogy(xs, surf_dens_y, marker='', linestyle='-', color='tab:blue')

plt.semilogy(-xs, surf_dens_x, marker='', linestyle='-', color='tab:red')
plt.semilogy(-xs, surf_dens_y, marker='', linestyle='-', color='tab:blue')

plt.xlim(-8, 8)
plt.ylim(1E-3, 1E1)

plt.xlabel(r"$R$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

In [ ]:
def density_on_grid(density, grid_lim=(-10,10), ngrid=128, args=()):
    grid = np.linspace(grid_lim[0], grid_lim[1], ngrid)
    xyz = np.vstack(map(np.ravel, np.meshgrid(grid,grid,grid)))

    val = np.zeros((ngrid*ngrid*ngrid,))
    val = density(xyz[0], xyz[1], xyz[2], *args)
    val[np.isnan(val)] = val[np.isfinite(val)].max()
    
    gridx = xyz[0].reshape(ngrid,ngrid,ngrid)[:,:,0]
    gridy = xyz[1].reshape(ngrid,ngrid,ngrid)[:,:,0]
    
    return gridx, gridy, val

In [ ]:
ngrid = 128
xx,yy,dens3d = density_on_grid(density_arr, ngrid=ngrid, args=args)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))

dens2d = dens3d.reshape(ngrid,ngrid,ngrid).sum(axis=2)
axes[0].contour(xx, yy, dens2d,
                levels=np.linspace(0.01, 10, 16), 
                colors='k', alpha=0.75)

dens2d = dens3d.reshape(ngrid,ngrid,ngrid).sum(axis=0)
axes[1].contour(xx, yy, dens2d.T,
                levels=np.linspace(0.01, 10, 16),
                colors='k', alpha=0.75)

fig.suptitle('True density', y=0.96, fontsize=22)

---


# Basis Function Expansion

Now we compute SCF expansion coefficients for this density distribution. Ultimately, we'll construct models that are truncated at the corotation radius, but to determine the corotation we'll use the un-truncated model.

To start with, we'll compute up to $n=10$ and $l=10$, but we might find below that we can truncate the expansion earlier.

In [ ]:
nmax = 10
lmax = 10
coeff = bscf.compute_coeffs(density,  nmax=nmax, lmax=lmax, 
                            M=1., r_s=1., skip_odd=True, args=args)
(S,Serr), (T,Terr) = coeff

In [ ]:
# np.save('n{0}_l{1}_dwek_G2.npy'.format(nmax, lmax), S)

Now we'll generate a fuckton of plots to compare the expansion to the true density at different truncation points (up to different intermediate nmax, lmax):

In [ ]:
xyz = np.zeros((3, 1024))
xyz[0] = np.logspace(-3, 1., xyz.shape[1])

xyz2 = np.zeros((3, 1024))
xyz2[1] = np.logspace(-3, 1., xyz2.shape[1])

# ---

fig,axes = plt.subplots(10, 5, figsize=(15,30), sharex=True, sharey=True)

for i,nmax in enumerate(range(1, 10+1, 1)):
    for j,lmax in enumerate(range(2, 10+1, 2)):
        _S = S[:nmax+1, :lmax+1, :lmax+1]
        pot = bscf.SCFPotential(m=1, r_s=1., Snlm=_S, Tnlm=np.zeros_like(_S))

        axes[i,j].semilogy(xyz[0], analytic_dens_x, marker='', linestyle='-')
        axes[i,j].semilogy(xyz[0], pot.density(xyz), marker='', linestyle='-', color='r', alpha=0.4, lw=5)
        
        if j == 0:
            axes[i,j].set_ylabel(r"$\rho(r)$")
            
        if i == 4:
            axes[i,j].set_xlabel(r"$r$ [kpc]")
            
        axes[i,j].text(1, 5, r'$n\leq{0}$, $l,m\leq{1}$'.format(nmax, lmax), 
                       fontsize=18)
        
axes[0,0].set_xlim(xyz[0].min(), xyz[0].max())
axes[0,0].set_ylim(1E-6, 1E2)

fig.tight_layout()

# ---

fig,axes = plt.subplots(10, 5, figsize=(15,30), sharex=True, sharey=True)

for i,nmax in enumerate(range(1, 10+1, 1)):
    for j,lmax in enumerate(range(2, 10+1, 2)):
        _S = S[:nmax+1, :lmax+1, :lmax+1]
        pot = bscf.SCFPotential(m=1, r_s=1., Snlm=_S, Tnlm=np.zeros_like(_S))

        axes[i,j].semilogy(xyz2[1], analytic_dens_y, marker='', linestyle='-')
        axes[i,j].semilogy(xyz2[1], pot.density(xyz2), marker='', linestyle='-', color='r', alpha=0.4, lw=5)
        
        if j == 0:
            axes[i,j].set_ylabel(r"$\rho(r)$")
            
        if i == 4:
            axes[i,j].set_xlabel(r"$r$ [kpc]")
            
        axes[i,j].text(1, 5, r'$n\leq{0}$, $l,m\leq{1}$'.format(nmax, lmax), 
                       fontsize=18)
        
axes[0,0].set_xlim(xyz[0].min(), xyz[0].max())
axes[0,0].set_ylim(1E-6, 1E2)

fig.tight_layout()

From this, it looks like $n_{\rm max} = 6$, $l_{\rm max} = 6$ would work, but let's see if it's computationally feasible compared to (3,6) as Wang & Zhao did:

In [ ]:
p1 = bscf.SCFPotential(m=1., r_s=1., 
                       Snlm=S[:6+1,:6+1,:6+1], Tnlm=np.zeros_like(S[:6+1,:6+1,:6+1]))

p2 = bscf.SCFPotential(m=1., r_s=1., 
                       Snlm=S[:3+1,:6+1,:6+1], Tnlm=np.zeros_like(S[:3+1,:6+1,:6+1]))

arr = np.array([[8., 0, 0]])
t = np.array([0.])
%timeit p1._gradient(arr, t)
%timeit p2._gradient(arr, t)

So it's not even a factor of 2 worse - I think we just go with the higher-order expansion then.

Let's now compare the surface density to the analytic:

In [ ]:
# pot = p1
pot = bscf.SCFPotential(m=1., r_s=1., Snlm=S, Tnlm=np.zeros_like(S))

In [ ]:
def surf_dens_helper2(z, R, pot, axis=0):
    q = np.array([0, 0, z])
    q[axis] = R
    return pot.density(q).value[0]

In [ ]:
surf_dens_x_scf = np.zeros_like(xs)
surf_dens_y_scf = np.zeros_like(xs)

for i,x in enumerate(xs):
    _surf_dens,_ = quad(surf_dens_helper2, 
                        -10, 10, 
                        args=(x, pot))
    surf_dens_x_scf[i] = _surf_dens
    
    _surf_dens,_ = quad(surf_dens_helper2, 
                        -10, 10, 
                        args=(x, pot, 1))
    surf_dens_y_scf[i] = _surf_dens

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5), sharex=True, sharey=True)

axes[0].semilogy(xs, surf_dens_x, marker='', linestyle='-')
axes[0].semilogy(xs, surf_dens_x_scf, marker='', linestyle='-', color='r', alpha=0.4, lw=5)

axes[1].semilogy(xs, surf_dens_y, marker='', linestyle='-')
axes[1].semilogy(xs, surf_dens_y_scf, marker='', linestyle='-', color='r', alpha=0.4, lw=5)

axes[0].set_xlim(xs.min(), 7.5)
axes[0].set_ylim(1E-4, 2E0)

axes[0].set_xlabel(r"$R$ [kpc]")
axes[1].set_xlabel(r"$R$ [kpc]")
axes[0].set_ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

Potential contours:

In [ ]:
grid = np.linspace(-10, 10, 256)

fig,axes = plt.subplots(1, 2, figsize=(12,6))
_ = pot.plot_contours(grid=(grid, grid, 0), ax=axes[0])
_ = pot.plot_contours(grid=(grid, 0, grid), ax=axes[1])
axes[0].axvline(5)
axes[0].axvline(-5)
axes[0].axhline(5)
axes[0].axhline(-5)

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].contourf(xx, yy, dens3d.sum(axis=2))
axes[1].contourf(xx, yy, dens3d.sum(axis=0).T)

In [ ]:
(np.abs(pot.parameters['Snlm'])>1E-10).sum()

In [ ]:
import h5py

In [ ]:
key = str(hash((6,6,0.)))

In [ ]:
with h5py.File('../barchaos/potential/data/coeffs.hdf5', 'a') as f:
    d = f.create_dataset(name=key, data=S[:6+1,:6+1,:6+1])
    d.attrs['Rmax'] = 100.